In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import time
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten,Dense,Activation,GlobalAveragePooling2D

import warnings
warnings.filterwarnings("ignore")


In [ ]:
class EyeDiseaseDataset:
   def __init__(self, dataDir):
        self.data_dir = dataDir

   def dataPaths(self):
        filepaths = []
        labels = []
        folds = os.listdir(self.data_dir)
        for fold in folds:
            foldPath = os.path.join(self.data_dir, fold)
            filelist = os.listdir(foldPath)
            for file in filelist:
                fpath = os.path.join(foldPath, file)
                filepaths.append(fpath)
                labels.append(fold)
        return filepaths, labels

   def dataFrame(self, files, labels):
        Fseries = pd.Series(files, name='filepaths')
        Lseries = pd.Series(labels, name='labels')
        return pd.concat([Fseries, Lseries], axis=1)

   def split_(self):
        files, labels = self.dataPaths()
        df = self.dataFrame(files, labels)
        strat = df['labels']
        trainData, dummyData = train_test_split(df, train_size=0.8, shuffle=True, random_state=42, stratify=strat)
        strat = dummyData['labels']
        validData, testData = train_test_split(dummyData, train_size=0.5, shuffle=True, random_state=42, stratify=strat)
        return trainData, validData, testData

In [ ]:
dataDir='/content/gdrive/MyDrive/ObjectDetection/EyesDeaseses/dataset'



In [ ]:
dataSplit = EyeDiseaseDataset(dataDir)
train_data, valid_data, test_data = dataSplit.split_()
# !ls '/content/gdrive/MyDrive/ObjectDetection/EyesDeaseses/dataset/cataract'



# Function for data Augmentation

In [ ]:
def augment_data(train_df,valid_df,test_df):
  img_size =(256,256)
  channels =3
  color ='rgb'

  train_datagen =tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=30,
                horizontal_flip =True,
                vertical_flip =True,
                brightness_range=[0.5, 1.5])

  valid_test_datagen =tf.keras.preprocessing.image.ImageDataGenerator()

  train_generator= train_datagen.flow_from_dataframe(
           train_df,
           x_col ='filepaths',
           y_col ='labels',
           target_size =img_size,
           color_mode= color,
           batch_size =channels,
           shuffle =True,
           class_mode ='categorical'
       )
  print("Shape of augmented training images:",train_generator.image_shape)

  valid_generator =valid_test_datagen.flow_from_dataframe(
           valid_df,
           x_col ='filepaths',
           y_col ='labels',
           target_size =img_size,
           color_mode= color,
           batch_size =channels,
           shuffle =False,
           class_mode ='categorical'
  )
  print("Shape of augmented valid images:",valid_generator.image_shape)

  test_generator =valid_test_datagen.flow_from_dataframe(
           test_df,
           x_col ='filepaths',
           y_col ='labels',
           target_size =img_size,
           color_mode= color,
           batch_size =channels,
           shuffle =False,
           class_mode ='categorical'
  )
  print("Shape of augmented test images:",test_generator.image_shape)

  return train_generator,test_generator,valid_generator


In [ ]:
train_generator,test_generator,valid_generator =augment_data(train_data,test_data,valid_data)

Found 3373 validated image filenames belonging to 4 classes.
Shape of augmented training images: (256, 256, 3)
Found 422 validated image filenames belonging to 4 classes.
Shape of augmented valid images: (256, 256, 3)
Found 422 validated image filenames belonging to 4 classes.
Shape of augmented test images: (256, 256, 3)


# Download and compile the model

In [ ]:
from tensorflow.keras.applications import EfficientNetB3

In [ ]:
classes = len(list(train_generator.class_indices.keys()))
base_model =EfficientNetB3(
            weights ='imagenet',
            include_top =False,
            input_shape =(256,256,3)
)

for layer in base_model.layers:
  layer.trainable=False

x= base_model.output
x=GlobalAveragePooling2D()(x)
x =Dense(256,activation='relu')(x)

predictions= Dense(classes,activation='softmax')(x)
model =Model(inputs=base_model.input,outputs=predictions)
optimizer =tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer,loss='categorical_crossentropy',metrics=['accuracy'])


# Fit the model

In [ ]:
history= model.fit(
    train_generator,
    epochs =25,
    validation_data =valid_generator
)

Epoch 1/25
1125/1125 [==============================] - 871s 762ms/step - loss: 0.6301 - accuracy: 0.7504 - val_loss: 0.5135 - val_accuracy: 0.7915
Epoch 2/25
1125/1125 [==============================] - 902s 802ms/step - loss: 0.4614 - accuracy: 0.8168 - val_loss: 0.7004 - val_accuracy: 0.7393
Epoch 3/25
1125/1125 [==============================] - 759s 675ms/step - loss: 0.3966 - accuracy: 0.8423 - val_loss: 0.5445 - val_accuracy: 0.8033
Epoch 4/25
1125/1125 [==============================] - 755s 670ms/step - loss: 0.3948 - accuracy: 0.8429 - val_loss: 0.3857 - val_accuracy: 0.8578
Epoch 5/25
1125/1125 [==============================] - 742s 660ms/step - loss: 0.3689 - accuracy: 0.8535 - val_loss: 0.3717 - val_accuracy: 0.8460
Epoch 6/25
1125/1125 [==============================] - 746s 663ms/step - loss: 0.3286 - accuracy: 0.8684 - val_loss: 0.3084 - val_accuracy: 0.8815
Epoch 7/25
1125/1125 [==============================] - 753s 669ms/step - loss: 0.3160 - accuracy: 0.8725 - val_

# Plot the accuracy and loss

In [ ]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
print("Training Accuracy:", train_accuracy[-1])
print("Validation Accuracy:", val_accuracy[-1])
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


NameError: name 'history' is not defined

# Affichage des images réelles et celles predites

In [ ]:
def plot_actuel_vs_predicted(model,test_data,num_samples=3):
  test_images,test_labels =next(iter(test_data))
  predictions =model.predict(test_images)
  class_labels =list(train_generator.class_indices.keys())
  sample_indices =np.random.choice(range(len(test_images)),num_samples,replace =False)
  for i in sample_indices:
    actual_label =class_labels[np.argmax(test_labels)]
    predicted_label= class_labels[np.argmax(predictions[i])]
    plt.figure(figsize=(8,4))

    plt.subplot(1,2,1)
    plt.imshow(test_images[i].astype(np.uint8))
    plt.title(f'Réelle:{actual_label}')
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(test_images[i].astype(np.uint8))
    plt.title(f'Predite:{predicted_label}')
    plt.axis('off')

    plt.show()

